# CNC Mill Tool Wear
- https://www.kaggle.com/shasun/tool-wear-detection-in-cnc-mill
- Predict "Machining_Process"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Read "train.csv"

In [ ]:
dataFolder = "./Data_CNC/"

In [ ]:
experiment_result = pd.read_csv(dataFolder + "train.csv")
print(f'train_XX.csv : {experiment_result.shape}')
experiment_result.head(3)

In [ ]:
experiment_result.info()

In [ ]:
experiment_result['passed_visual_inspection']

## Deal with NaN

In [ ]:
experiment_result['passed_visual_inspection'] = experiment_result['passed_visual_inspection'].fillna('no')
experiment_result['passed_visual_inspection']

## Read "experiment_XX.csv"

In [ ]:
experiment_tmp = pd.read_csv(dataFolder + "experiment_01.csv")
print(f'experiment_XX.csv : {experiment_tmp.shape}')
print(experiment_tmp.columns)
experiment_tmp.head(3)

In [ ]:
experiment_tmp.info()

In [ ]:
filenames = [f'experiment_0{i}.csv' if i < 10 else f'experiment_{i}.csv' for i in range(1,19)]
print(filenames)

In [ ]:
frames = []
for (idx, filename) in zip(range(1,19), filenames):
    #load files
    frame = pd.read_csv(dataFolder + filename)

    #load each experiment result row
    filt = experiment_result['No'] == idx
    exp_result_row = experiment_result[filt]
    frame['exp_num'] = idx

    #add experiment settings to features
    frame['material'] = exp_result_row.iloc[0]['material']
    frame['feedrate'] = exp_result_row.iloc[0]['feedrate']
    frame['clamp_pressure'] = exp_result_row.iloc[0]['clamp_pressure']
    
    #add experiment result to features
    frame['tool_condition'] = exp_result_row.iloc[0]['tool_condition']
    frame['machining_finalized'] = exp_result_row.iloc[0]['machining_finalized']
    frame['passed_visual_inspection'] = exp_result_row.iloc[0]['passed_visual_inspection']

    frames.append(frame)

df = pd.concat(frames, ignore_index = True)
display(df.head(3))
print(df.shape)

In [ ]:
df.info()

## Inspect number columns

In [ ]:
cols = list(df.columns)
catCols = ['Machining_Process','machining_finalized','passed_visual_inspection','tool_condition','material','exp_num']

numCols = cols
for catCol in catCols:
    numCols.remove(catCol) #Mutatble
print(numCols)

# Shorter syntax and non-mutable
#numCols = [item for item in cols if item not in catCols]

In [ ]:
dfNum = df[numCols]
dfNum.hist(figsize=(20,20))
display()

## Inspect catagorical columns

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(18,10),gridspec_kw={'hspace': 0.5, 'wspace': 0.2})
idx = 0
for row in range(0,2):
    for col in range(0,3):
        catCol = catCols[idx]
        df[catCol].value_counts().plot.bar(ax=ax[row][col])
        ax[row][col].set_title(catCol)
        idx = idx + 1

## Drop columns

In [ ]:
catColsDrop = [item for item in catCols if item != 'Machining_Process']
df2 = df.drop(columns=catColsDrop)

In [ ]:
df2.info()

## Replacing data

In [ ]:
display(df2['Machining_Process'].value_counts())

In [ ]:
df2 = df2.replace({'Machining_Process': {'Starting':'Prep','end':'End'}})
df2['Machining_Process'].value_counts().plot.bar()
display()

## Data Encoding

### Testing

In [ ]:
temp = df[['tool_condition']].copy()
display(temp)
temp['tool_condition'].value_counts()

In [ ]:
le = LabelEncoder()
le.fit(df['tool_condition'])
temp['tool_condition'] = le.transform(temp['tool_condition'])
display(temp)
temp['tool_condition'].value_counts()

In [ ]:
temp['tool_condition'] = le.inverse_transform(temp['tool_condition'])
temp['tool_condition'].value_counts()

### Applying to actual data

In [ ]:
le = LabelEncoder()
df2['Machining_Process'] = le.fit_transform(df2['Machining_Process'])
df2['Machining_Process'].value_counts()

In [ ]:
df2.info()

## Save processed data
- CSV
- Pickle

In [ ]:
saveFolder = 'Data_CNC_processed'
filename = 'pred_machining_process'
df2.to_csv(f'{saveFolder}/{filename}.csv', index=False)

In [ ]:
compression = 'gzip'
df2.to_pickle(f'{saveFolder}/{filename}.pkl',compression=compression)

In [ ]:
df3 = pd.read_csv(f'{saveFolder}/{filename}.csv')
df3.head(3)

In [ ]:
df4 = pd.read_pickle(f'{saveFolder}/{filename}.pkl',compression=compression)
df4.head(3)

## Machining learning (finally)

In [ ]:
colsX = [item for item in df2.columns if item not in 'Machining_Process'] 
X = df2[colsX].values
y = df2['Machining_Process'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

In [ ]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
eta0 = 0.01
random_state=1
n_iter_no_change=10

In [ ]:
ppn = Perceptron(
    eta0=eta0,
    random_state=random_state,
    verbose=1,
    n_iter_no_change=n_iter_no_change,
    )

In [ ]:
# Training
ppn.fit(X_train_std, y_train)

In [ ]:
# Prediction
y_pred = ppn.predict(X_test_std)

In [ ]:
# Misclassification from the test samples
sumMiss = (y_test != y_pred).sum()

In [ ]:
# Accuracy score from the test samples
accuracyScore = accuracy_score(y_test, y_pred)

In [ ]:
print(f"Misclassified examples: {sumMiss}")
print(f"Accuracy score: {accuracyScore}")

In [ ]:
y_test.shape